In [ ]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import random
import re
import base64
from io import BytesIO
from PIL import Image
from IPython.display import Audio, display
import speech_recognition as sr #requires pip install speechrecognition AND pip install pyaudio

In [ ]:
load_dotenv()

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins with {openai_api_key[:8]}")
else:
    print("OpenAI API Key? As if!")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()

In [ ]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so. "
system_message += "You can book flights directly. "
system_message += "You can generate beautiful artistic renditions of the cities we fly to."

In [ ]:
# ticket price function

#spelled-out currency notation for better tts rendition
ticket_prices = {"valletta": "799 Dollars", "turin": "899 Dollars", "sacramento": "1400 Dollars", "montreal": "499 Dollars"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

In [ ]:
# travel booking function

def book_flight(destination_city):
    booking_code = ''.join(random.choice('0123456789BCDFXYZ') for i in range(2)) + ''.join(random.choice('012346789HIJKLMNOPQRS') for i in range(2)) + ''.join(random.choice('0123456789GHIJKLMNUOP') for i in range(2))
    print(f"Booking code {booking_code} generated for flight to {destination_city}.")
    
    return booking_code

In [ ]:
# verify if booking code is valid (i.e. follows the pattern)

def check_code(code):
    valid = "valid" if re.match("^[0123456789BCDFXYZ]{2}[012346789HIJKLMNOPQRS]{2}[0123456789GHIJKLMNUOP]{2}$", code) != None else "not valid"
    print(f"Code checker called for code {code}, which is {valid}.")
    return re.match("^[0123456789BCDFXYZ]{2}[012346789HIJKLMNOPQRS]{2}[0123456789GHIJKLMNUOP]{2}$", code) != None

In [ ]:
# make a nice preview of the travel destination

def artist(city):
    image_response = openai.images.generate(
        model="dall-e-3",
        prompt=f"Make an image in the style of a vibrant, artistically filtered photo that is a collage of the best sights and views in {city}.",
        size="1024x1024",
        n=1,
        response_format="b64_json",
    )
    image_base64 = image_response.data[0].b64_json
    image_data = base64.b64decode(image_base64)
    img = Image.open(BytesIO(image_data))

    img.save("img001.png") #make them 4 cents count! .save is from PIL library, btw
    
    return img

In [ ]:
price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [ ]:
booking_function = {
    "name": "book_flight",
    "description": "Call this whenever you have to book a flight. Give it the destination city and you will get a booking code. Tell the customer \
that the flight is booked and give them the booking code obtained through this function. Never give any other codes to the customer.",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to book their flight to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [ ]:
artist_function = {
    "name": "artist",
    "description": "Call this whenever you need to generate a picture, photo, or graphic impression of a city.",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "The city of which an image is to be generated",
            },
        },
        "required": ["city"],
        "additionalProperties": False
    }
}

In [ ]:
codecheck_function = {
    "name": "check_code",
    "description": "Call this whenever you need to verify if a booking code for a flight (also called 'flight code', 'booking reference', \
or variations thereof) is valid.",
    "parameters": {
        "type": "object",
        "properties": {
            "code": {
                "type": "string",
                "description": "The code that you or the user needs to verify",
            },
        },
        "required": ["code"],
        "additionalProperties": False
    }
}

In [ ]:
# List of tools:

tools = [{"type": "function", "function": price_function}, {"type": "function", "function": booking_function}, {"type": "function", "function": codecheck_function}, {"type": "function", "function": artist_function}]

In [ ]:
# speech recognition (STT) by Google

r = sr.Recognizer()

def speech_to_text():
    try:
        with sr.Microphone() as source:
            r.adjust_for_ambient_noise(source, duration=0.2)
            speech = r.listen(source, 10, 5) #timeout of 10 seconds, listen for 5
            text = r.recognize_google(speech)
            print(f"STT heard: \"{text}\"")
            return text

    # sometimes, this STT fails. You'll see "..." as your input. Just try again even w/o re-starting Gradio.
    except sr.RequestError as e:
        print(f"Could not request results; {0}".format(e))
        return "…"
    except sr.UnknownValueError:
        print("An unknown error occurred")
        return "…"
    except sr.WaitTimeoutError:
        print("Wait timed out")
        return "…"

In [ ]:
# TTS by OpenAI

def talker(message):
    response = openai.audio.speech.create(
        model="tts-1",
        voice="onyx",
        input=message)

    audio_stream = BytesIO(response.content)
    output_filename = "output_audio.mp3"
    with open(output_filename, "wb") as f:
        f.write(audio_stream.read())

    # Play the generated audio
    display(Audio(output_filename, autoplay=True))

In [ ]:
def chat(history):
    messages = [{"role": "system", "content": system_message}] + history
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    image = None
    
    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        responses = handle_tool_call(message)[0]
        image = handle_tool_call(message)[1]
        messages.append(message)
        for response in responses:
            messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
        
    reply = response.choices[0].message.content

    # comment in if you want the replies read out to you. Mind the price!
    #talker(reply) #current cost: $0.015 per 1000 characters (not tokens!)
    
    history += [{"role": "assistant", "content": reply}]
    
    return history, image

In [ ]:
def handle_tool_call(message):
    responses = []
    image = None
    
    for tool_call in message.tool_calls:
        arguments = json.loads(tool_call.function.arguments)
        indata = arguments[list(arguments.keys())[0]] # works for now because we only have one argument in each of our functions
        function_name = tool_call.function.name
        if function_name == 'get_ticket_price':
            outdata = get_ticket_price(indata)
            input_name = "destination city"
            output_name = "price"
        elif function_name == 'book_flight':
            outdata = book_flight(indata)
            input_name = "destination city"
            output_name = "booking code"
        elif function_name == "check_code":
            outdata = check_code(indata)
            input_name = "booking code"
            output_name = "validity"
        elif function_name == "artist":
            image = artist(indata)
            outdata = f"artistic rendition of {indata}"
            input_name = "city"
            output_name = "image"

        responses.append({
            "role": "tool",
            "content": json.dumps({input_name: indata, output_name: outdata}),
            "tool_call_id": tool_call.id
        })

    return responses, image

In [ ]:
## Gradio with 'fancy' buttons. Claude explained this css business to me, and geeksforgeeks.
## see week2/community-contributions/day5_Careerhelper.ipynb for a much more competent version of this.

with gr.Blocks(
    css="""
    .red-button {
        background-color: darkred !important;
        border-color: red !important;
    }
    .blue-button {
        background-color: darkblue !important;
        border-color: blue !important;
    }
    """
) as ui:
    with gr.Row():
        chatbot = gr.Chatbot(height=500, type="messages")
        image_output = gr.Image(height=500)
    with gr.Row():
        entry = gr.Textbox(label="Chat with our AI Assistant:")
    with gr.Row():
            speak = gr.Button(value="Speak to our AI Assistant", elem_classes="blue-button")
            clear = gr.Button(value="Clear Chat", elem_classes="red-button")

    def do_entry(message, history):
        history += [{"role":"user", "content":message}]
        return "", history

    def listen(history):
        message = speech_to_text()
        history += [{"role":"user", "content":message}]
        return history

    entry.submit(do_entry, inputs=[entry, chatbot], outputs=[entry, chatbot]).then(
        chat, inputs=chatbot, outputs=[chatbot, image_output]
    )
    
    clear.click(lambda: None, inputs=None, outputs=chatbot, queue=False)
    
    speak.click(listen, inputs=chatbot, outputs=chatbot, queue=False).then(
        chat, inputs=chatbot, outputs=[chatbot, image_output]
    )

ui.launch(inbrowser=True)